<a href="https://colab.research.google.com/github/daycardoso/PredictCost/blob/main/regress%C3%A3o_linear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
import pandas as pd
import cudf
import cupy as cp
from cuml.linear_model import LinearRegression as cuLR
from sklearn.linear_model import LinearRegression as skLR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# =====================
# 1. Carregamento dos Dados
# =====================
try:
    df_cudf = cudf.read_csv('/content/drive/MyDrive/Mestrado/Trabalho ML Mestrado 01-2025/df_unificado.csv')
except:
    df_pd = pd.read_csv('/content/drive/MyDrive/Mestrado/Trabalho ML Mestrado 01-2025/df_unificado.csv')
    df_cudf = cudf.from_pandas(df_pd)

# Conversão para float32
for col in df_cudf.select_dtypes(include=['float64']).columns:
    df_cudf[col] = df_cudf[col].astype(cp.float32)

X = df_cudf.iloc[:, :-1]
y = df_cudf.iloc[:, -1]

# =====================
# 2. Divisão treino/teste
# =====================
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# =====================
# 3. Treinamento do modelo
# =====================
try:
    linear_regression_model = cuLR()
    linear_regression_model.fit(X_train, y_train)
    y_pred = linear_regression_model.predict(X_test)
except Exception as e:
    print("cuML falhou, usando scikit-learn:", e)
    X_train_pd = X_train.to_pandas() if hasattr(X_train, "to_pandas") else X_train
    y_train_pd = y_train.to_pandas() if hasattr(y_train, "to_pandas") else y_train
    X_test_pd = X_test.to_pandas() if hasattr(X_test, "to_pandas") else X_test

    linear_regression_model = skLR()
    linear_regression_model.fit(X_train_pd, y_train_pd)
    y_pred = linear_regression_model.predict(X_test_pd)

# =====================
# 4. Avaliação
# =====================
y_test_cpu = y_test.to_pandas() if hasattr(y_test, "to_pandas") else y_test
y_pred_cpu = y_pred.to_pandas() if hasattr(y_pred, "to_pandas") else y_pred

print("R²:", r2_score(y_test_cpu, y_pred_cpu))
print("MSE:", mean_squared_error(y_test_cpu, y_pred_cpu))
print("MAE:", mean_absolute_error(y_test_cpu, y_pred_cpu))


/usr/local/lib/python3.11/dist-packages/cudf/core/dataframe.py:7737: FutureWarning: Using `__dataframe__` is deprecated
  warnings.warn("Using `__dataframe__` is deprecated", FutureWarning)


R²: 0.712709550510821
MSE: 0.001094289967591171
MAE: 0.02773324448692172


In [25]:
import numpy as np
# Import garbage collector for memory cleanup
import gc

from sklearn.metrics import (
    mean_squared_error, r2_score, median_absolute_error,
    max_error, explained_variance_score
)
from sklearn.model_selection import KFold
from cuml.linear_model import LinearRegression as cuLR
from sklearn.linear_model import LinearRegression as skLR


def nested_cv_linear_evaluation(X, y, model_class, outer_folds=5, random_state=42):
    # Renamed the function to accept a model_class argument
    outer_cv = KFold(n_splits=outer_folds, shuffle=True, random_state=random_state)

    mse_list = []
    r2_list = []
    medae_list = []
    maxe_list = []
    evs_list = []

    for fold_idx, (train_idx, test_idx) in enumerate(outer_cv.split(X), 1):
        X_tr, X_te = X.iloc[train_idx], X.iloc[test_idx]
        y_tr, y_te = y.iloc[train_idx], y.iloc[test_idx]

        # Instantiate the model using the provided model_class
        model = model_class()
        model.fit(X_tr, y_tr)
        y_pred = model.predict(X_te)

        # Avaliação das métricas
        # Ensure the predictions and true values are on the CPU for sklearn metrics
        y_te_cpu = y_te.to_pandas() if hasattr(y_te, "to_pandas") else y_te
        y_pred_cpu = y_pred.to_pandas() if hasattr(y_pred, "to_pandas") else y_pred

        mse  = float(mean_squared_error(y_te_cpu, y_pred_cpu))
        r2   = float(r2_score(y_te_cpu, y_pred_cpu))
        med  = float(median_absolute_error(y_te_cpu, y_pred_cpu))
        maxe = float(max_error(y_te_cpu, y_pred_cpu))
        evs  = float(explained_variance_score(y_te_cpu, y_pred_cpu))


        # Armazenamento
        mse_list.append(mse)
        r2_list.append(r2)
        medae_list.append(med)
        maxe_list.append(maxe)
        evs_list.append(evs)

        print(
            f"[Linear] Fold {fold_idx}/{outer_folds} - "
            f"MSE: {mse:.5f} | R²: {r2:.4f} | MedAE: {med:.4f} | "
            f"MaxErr: {maxe:.4f} | EVS: {evs:.4f}"
        )

        # Limpeza
        del model, X_tr, X_te, y_tr, y_te, y_pred, y_te_cpu, y_pred_cpu # Added y_te_cpu and y_pred_cpu to cleanup
        gc.collect()
        cp.get_default_memory_pool().free_all_blocks()

    # Resultados médios
    metrics = {
        "MSE": np.mean(mse_list),
        "R2": np.mean(r2_list),
        "Median AE": np.mean(medae_list),
        "Max Error": np.mean(maxe_list),
        "Explained Variance": np.mean(evs_list)
    }

    print("\n[Métricas Médias - Regressão Linear]")
    for k, v in metrics.items():
        print(f"{k}: {v:.4f}")

    return metrics

In [26]:
# === Dados já carregados como X, y ===

# Choose which model class to use based on availability or preference
# You might want to add logic here to choose between cuLR and skLR
# based on whether cuML failed in the previous cell.
# For this example, let's assume cuML is available and works.
model_to_evaluate = cuLR

# Avaliação do modelo linear com Nested CV (sem tuning)
# Pass the model class to the function
linear_metrics = nested_cv_linear_evaluation(X, y, model_to_evaluate)
print(f"\nMSE médio Linear Regression (Nested CV): {linear_metrics['MSE']:.4f}")

# Definição do Random Forest com melhores hiperparâmetros encontrados antes
# rf_cls = lambda: cuRF(random_state=42, n_streams=1, **chosen_rf)

# Comparação estatística: Linear vs RF
# t_stat, p_val = five_two_cv_ttest_gpu(linear_regression_model, rf_cls, X_cudf, y_cudf)
# print(f"\n5×2-cv t-test (Linear vs RF): t = {t_stat:.4f}, p = {p_val:.4f}")

/usr/local/lib/python3.11/dist-packages/cudf/core/dataframe.py:7737: FutureWarning: Using `__dataframe__` is deprecated
  warnings.warn("Using `__dataframe__` is deprecated", FutureWarning)


[Linear] Fold 1/5 - MSE: 0.00109 | R²: 0.7127 | MedAE: 0.0256 | MaxErr: 0.3347 | EVS: 0.7127


/usr/local/lib/python3.11/dist-packages/cudf/core/dataframe.py:7737: FutureWarning: Using `__dataframe__` is deprecated
  warnings.warn("Using `__dataframe__` is deprecated", FutureWarning)


[Linear] Fold 2/5 - MSE: 0.00110 | R²: 0.7124 | MedAE: 0.0257 | MaxErr: 0.2718 | EVS: 0.7124


/usr/local/lib/python3.11/dist-packages/cudf/core/dataframe.py:7737: FutureWarning: Using `__dataframe__` is deprecated
  warnings.warn("Using `__dataframe__` is deprecated", FutureWarning)


[Linear] Fold 3/5 - MSE: 0.00109 | R²: 0.7128 | MedAE: 0.0256 | MaxErr: 0.2082 | EVS: 0.7128


/usr/local/lib/python3.11/dist-packages/cudf/core/dataframe.py:7737: FutureWarning: Using `__dataframe__` is deprecated
  warnings.warn("Using `__dataframe__` is deprecated", FutureWarning)


[Linear] Fold 4/5 - MSE: 0.00109 | R²: 0.7123 | MedAE: 0.0256 | MaxErr: 0.2180 | EVS: 0.7123


/usr/local/lib/python3.11/dist-packages/cudf/core/dataframe.py:7737: FutureWarning: Using `__dataframe__` is deprecated
  warnings.warn("Using `__dataframe__` is deprecated", FutureWarning)


[Linear] Fold 5/5 - MSE: 0.00109 | R²: 0.7126 | MedAE: 0.0256 | MaxErr: 0.2560 | EVS: 0.7126

[Métricas Médias - Regressão Linear]
MSE: 0.0011
R2: 0.7126
Median AE: 0.0256
Max Error: 0.2577
Explained Variance: 0.7126

MSE médio Linear Regression (Nested CV): 0.0011


In [27]:
import time

def measure_inference_time(model, X_test, n_repeats=5):
    """
    Mede o tempo médio de inferência por amostra.
    Repete a medição 'n_repeats' vezes para reduzir ruído.
    """
    total_times = []
    for _ in range(n_repeats):
        start = time.perf_counter()
        y_pred = model.predict(X_test)
        end = time.perf_counter()
        total_times.append(end - start)

    avg_total_time = sum(total_times) / len(total_times)
    time_per_sample = avg_total_time / len(X_test)
    return time_per_sample


In [28]:
# Após modelo.fit(...)
X_test_eval = X_test.to_pandas() if hasattr(X_test, "to_pandas") else X_test
inference_time = measure_inference_time(linear_regression_model, X_test_eval)
print(f"Tempo médio de inferência por amostra (Linear): {inference_time * 1000:.9f} ms")


Tempo médio de inferência por amostra (Linear): 0.000040349 ms


In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import cudf
import cupy as cp
import numpy as np



# Conversão para float32
for col in df_cudf.select_dtypes(include=['float64']).columns:
    df_cudf[col] = df_cudf[col].astype(cp.float32)

X = df_cudf.iloc[:, :-1].to_pandas().values.astype(np.float32)
y = df_cudf.iloc[:, -1].to_pandas().values.astype(np.float32).reshape(-1, 1)

# ====================
# 2. Divisão treino/teste
# ====================
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tensores para GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
X_train_tensor = torch.from_numpy(X_train).to(device)
y_train_tensor = torch.from_numpy(y_train).to(device)
X_test_tensor  = torch.from_numpy(X_test).to(device)
y_test_tensor  = torch.from_numpy(y_test).to(device)

# ====================
# 3. Definição do MLP
# ====================
class SmallMLP(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        return self.model(x)

model = SmallMLP(X_train.shape[1]).to(device)

# ====================
# 4. Treinamento
# ====================
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()
n_epochs = 50
batch_size = 1024

for epoch in range(n_epochs):
    model.train()
    permutation = torch.randperm(X_train_tensor.size()[0])
    for i in range(0, X_train_tensor.size()[0], batch_size):
        indices = permutation[i:i+batch_size]
        batch_x, batch_y = X_train_tensor[indices], y_train_tensor[indices]

        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = loss_fn(outputs, batch_y)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}/{n_epochs} - Loss: {loss.item():.6f}")

# ====================
# 5. Avaliação
# ====================
model.eval()
with torch.no_grad():
    y_pred_tensor = model(X_test_tensor)
    y_pred = y_pred_tensor.cpu().numpy()
    y_true = y_test_tensor.cpu().numpy()

print("\n[Métricas - MLP]")
print("MSE:", mean_squared_error(y_true, y_pred))
print("R²:", r2_score(y_true, y_pred))

# ====================
# 6. Tempo de Inferência
# ====================
import time

with torch.no_grad():
    start = time.perf_counter()
    _ = model(X_test_tensor)
    torch.cuda.synchronize()  # garante sincronização GPU
    end = time.perf_counter()

inference_time = (end - start) / len(X_test_tensor)
print(f"Tempo médio de inferência por amostra: {inference_time * 1000:.6f} ms")


Epoch 1/50 - Loss: 0.001583
Epoch 2/50 - Loss: 0.000920
Epoch 3/50 - Loss: 0.001458
Epoch 4/50 - Loss: 0.000620
Epoch 5/50 - Loss: 0.000786
Epoch 6/50 - Loss: 0.000749
Epoch 7/50 - Loss: 0.000763
Epoch 8/50 - Loss: 0.000780
Epoch 9/50 - Loss: 0.000535
Epoch 10/50 - Loss: 0.000601
Epoch 11/50 - Loss: 0.000608
Epoch 12/50 - Loss: 0.000501
Epoch 13/50 - Loss: 0.000671
Epoch 14/50 - Loss: 0.000550
Epoch 15/50 - Loss: 0.000610
Epoch 16/50 - Loss: 0.000656
Epoch 17/50 - Loss: 0.000659
Epoch 18/50 - Loss: 0.000681
Epoch 19/50 - Loss: 0.000505
Epoch 20/50 - Loss: 0.000631
Epoch 21/50 - Loss: 0.000739
Epoch 22/50 - Loss: 0.000481
Epoch 23/50 - Loss: 0.000542
Epoch 24/50 - Loss: 0.000748
Epoch 25/50 - Loss: 0.000557
Epoch 26/50 - Loss: 0.000597
Epoch 27/50 - Loss: 0.000576
Epoch 28/50 - Loss: 0.000474
Epoch 29/50 - Loss: 0.000644
Epoch 30/50 - Loss: 0.000452
Epoch 31/50 - Loss: 0.000582
Epoch 32/50 - Loss: 0.000611
Epoch 33/50 - Loss: 0.000611
Epoch 34/50 - Loss: 0.000526
Epoch 35/50 - Loss: 0.0